# ABoffsets_LSSTComCam_c26202

Authors:  C. L. Adair, D. L. Tucker, with help from L. Jones, J. Carlin, and others

Created:  2024.11.15
Updated: 2025.01.29 (and it has cool stuff in here...I'm impressed with us)

## 1. Initial Setup...

In [1]:
%pwd

'/home/d/dltucker/WORK/GitHub/SP-1736/sciunit_photocalib/notebooks'

### 1.1 Import useful python packages

In [2]:
# Generic python packages
import pylab as plt
import numpy as np
import pandas as pd
import glob
import math
import os
import gc
import warnings

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay

# rubin_sim-related packages
import rubin_sim.phot_utils as pt
import syseng_throughputs as st
from rubin_sim.data import get_data_dir

# Astropy-related packages
from astropy import units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
import lsst.geom as geom

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

# Set filter warnings to "ignore" to avoid a lot of "logorrhea" to the screen:
warnings.filterwarnings("ignore")

/home/d/dltucker/WORK/GitHub/syseng_throughputs/syseng_throughputs/bandpassUtils.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


### 1.2 Include user input

In [3]:
# Which repo, collection, instrument, and skymap to use.
# See https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.-ComCam
# and https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs
#repo = 'embargo'
repo = '/repo/dp1'
collections = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'

instrument = 'LSSTComCam'
skymap_name = 'lsst_cells_v1'
day_obs_start = 20241101
day_obs_end = 20241231

plotCutouts = False

# Set environment variable to point to location of the rubin_sim_data 
#  (per Lynne Jones' Slack message on the #sciunit-photo-calib channel from 26 Nov 2024):
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Which CalSpec C26202 spectrum FITS files to to use?
sedfile_dict = {'stiswfcnic_007' : '~/Downloads/c26202_stiswfcnic_007.fits', 
                'mod_008'        : '~/Downloads/c26202_mod_008.fits'
               }

# RA, DEC of C26202 in degrees (from `/home/d/dltucker/DATA/SynthMags/synthMagColorList.lsst_v1.9.calspec_20240603.added_info.csv`):
raDeg = 53.136845833333325
decDeg = -27.86349444444444

# List of filters to examine
flist = ['u','g','r','i','z','y']

# Plot symbol colors to use for ugrizy
plot_filter_colors_white_background = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00', 'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

### 1.3 Define useful classes and functions

In [4]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [5]:
def cutout_im(butler, ra, dec, datasetType, visit, detector, cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a preliminary_visit_image at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)
    wcs = butler.get('%s.wcs' % datasetType,**dataId)
    xy = geom.PointI(wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get(datasetType, parameters=parameters, **dataId)

    return cutout_image

In [6]:
def warp_img(ref_img, img_to_warp, ref_wcs, wcs_to_warp):

    config = RegisterConfig()
    task = RegisterTask(name="register", config=config)
    warpedExp = task.warpExposure(img_to_warp, wcs_to_warp, ref_wcs,
                                  ref_img.getBBox())

    return warpedExp

In [7]:
def make_gif(frame_folder):
    frames = [Image.open(image) for image in sorted(glob.glob(f"{frame_folder}/*.png"))]
    frame_one = frames[0]
    frame_one.save("animation.gif", format="GIF", append_images=frames,
               save_all=True, duration=500, loop = 0)

## 2. Calculate Synthetic AB magnitudes for C26202, based on official filter bandpasses

### 2.1 Change detectors from (default) LSST to ComCam

In [8]:
defaultDirs = st.setDefaultDirs()
defaultDirs['detector'] = defaultDirs['detector'].replace('/joint_minimum', '/itl')
hardware, system = st.buildHardwareAndSystem(defaultDirs)


### 2.2 Calculate synthetic mags

In [9]:
mags = {}

# Loop through all SEDs in our sedfile dictionary
for sed_key in sedfile_dict:
    
    print(sed_key, sedfile_dict[sed_key])
    
    # Read the SED file associated with this SED
    sedfile = sedfile_dict[sed_key]
    seddata = fits.getdata(sedfile)

    # Transform the SED data into rubin_sim format
    wavelen = seddata['WAVELENGTH'] * u.angstrom.to(u.nanometer) # This is in angstroms - need in nanometers
    flambda = seddata['FLUX'] / (u.angstrom.to(u.nanometer)) # this is in erg/sec/cm^^2/ang but we want /nm     
    sed = pt.Sed(wavelen=wavelen, flambda=flambda)
    
    # Loop over the filters, calculating the synthetic mags for each filter for this SED
    mags[sed_key] = []
    for f in flist:
        # Append the synthetic mag for this filter to this mags list for this SED
        mags[sed_key].append(sed.calc_mag(system[f]))
    # Convert list of synthetic mags for this SED into a numpy array
    mags[sed_key] = np.array(mags[sed_key])
    
    

stiswfcnic_007 ~/Downloads/c26202_stiswfcnic_007.fits
mod_008 ~/Downloads/c26202_mod_008.fits


### 2.3 Convert mags numpy arrays into a pandas dataframe

In [10]:
df_mags = pd.DataFrame(mags, index=flist)
df_mags

,stiswfcnic_007,mod_008
u,17.572800,17.586964
g,16.691931,16.692687
r,16.362017,16.361654
i,16.260196,16.259542
z,16.243679,16.243690
y,16.238847,16.238887


## 3. Query USDF Butler for ComCam measurements of C26202

### 3.1 Instantiate Butler

In [11]:
butler = dafButler.Butler(repo, collections=collections)

### 3.2 Find all the `preliminary_visit_image`'s that overlap the sky position of C26202

#### 3.2.1 Find the `dataId`'s for all `preliminary_visit_image`'s in this repo/collection that overlap the RA, DEC of C26202

In [12]:
datasetRefs = butler.query_datasets("preliminary_visit_image", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

for i, ref in enumerate(datasetRefs):    
    print(i, ref.dataId)

print(f"\nFound {len(datasetRefs)} preliminary_visit_images")

0 {instrument: 'LSSTComCam', detector: 5, visit: 2024120500122, band: 'r', day_obs: 20241205, physical_filter: 'r_03'}
1 {instrument: 'LSSTComCam', detector: 5, visit: 2024120900310, band: 'r', day_obs: 20241209, physical_filter: 'r_03'}
2 {instrument: 'LSSTComCam', detector: 4, visit: 2024110800288, band: 'r', day_obs: 20241108, physical_filter: 'r_03'}
3 {instrument: 'LSSTComCam', detector: 5, visit: 2024110800251, band: 'r', day_obs: 20241108, physical_filter: 'r_03'}
4 {instrument: 'LSSTComCam', detector: 3, visit: 2024111900094, band: 'y', day_obs: 20241119, physical_filter: 'y_04'}
5 {instrument: 'LSSTComCam', detector: 0, visit: 2024111700154, band: 'r', day_obs: 20241117, physical_filter: 'r_03'}
6 {instrument: 'LSSTComCam', detector: 5, visit: 2024120900311, band: 'r', day_obs: 20241209, physical_filter: 'r_03'}
7 {instrument: 'LSSTComCam', detector: 3, visit: 2024120600097, band: 'r', day_obs: 20241206, physical_filter: 'r_03'}
8 {instrument: 'LSSTComCam', detector: 3, visit:

#### 3.2.2 Plot the cutouts for all these `preliminary_visit_image`'s

**Tina, could use add some code here from the DP02_04b_Intermediate_Butler_Queries tutorial notebook so we can view these `preliminary_visit_image` images?  Maybe something from Section 3.1 from that tutorial notebook.  It would be good to take a look at the individual images in case there are any weird `preliminary_visit_image` images that we should ignore.**

In [13]:
preliminary_visit_image = butler.get('preliminary_visit_image', dataId={'visit': 2024111100087, 'detector': 0})

In [14]:
preliminary_visit_image_info = preliminary_visit_image.getInfo()

In [15]:
visit_info = preliminary_visit_image_info.getVisitInfo()
summary_info = preliminary_visit_image_info.getSummaryStats()

In [16]:
summary_info

ExposureSummaryStats(version=0, psfSigma=2.5164382595440666, psfArea=98.97710104140349, psfIxx=6.12938463560872, psfIyy=6.573291725610319, psfIxy=-0.43607852596518165, ra=53.132176257380756, dec=-27.762129453534705, pixelScale=0.20041974721286201, zenithDistance=36.069318690012004, expTime=30.0, zeroPoint=32.03303835757499, skyBg=3888.5185546875, skyNoise=63.91310411363439, meanVar=4138.878924675822, raCorners=[53.2207680500666, 53.2868290939046, 53.04344393439571, 52.9777810042101], decCorners=[-27.623802003888713, -27.842878190635926, -27.90026929950566, -27.68116399706156], astromOffsetMean=0.009982084116100453, astromOffsetStd=0.005017610726637799, nPsfStar=78, psfStarDeltaE1Median=-0.000500578433275306, psfStarDeltaE2Median=0.0020652189850806912, psfStarDeltaE1Scatter=0.013239352696123394, psfStarDeltaE2Scatter=0.0170912067529773, psfStarDeltaSizeMedian=0.007213981549011983, psfStarDeltaSizeScatter=0.020557948910376093, psfStarScaledDeltaSizeScatter=0.00811480995209903, psfTraceRa

In [17]:
datasetType = 'preliminary_visit_image'
dataId = {'visit': 2024111100087, 'detector': 0}
preliminary_visit_image = butler.get(datasetType, dataId=dataId)

In [18]:
print(butler.registry.getDatasetType(datasetType))

DatasetType('preliminary_visit_image', {band, instrument, day_obs, detector, physical_filter, visit}, ExposureF)


In [19]:
if plotCutouts:
    fig = plt.figure()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(preliminary_visit_image.image)
    plt.show()

In [20]:
cutoutsize = 501 #Defining the size of the cutout box in pixels
visit = 2024111100087
detector = 0

In [21]:
cutout_preliminary_visit_image = cutout_im(butler, raDeg, decDeg, 'preliminary_visit_image', visit, detector, cutoutSideLength=cutoutsize)

In [22]:
if plotCutouts:
    fig = plt.figure()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(cutout_preliminary_visit_image.image)
    plt.show()

#### 3.2.3 Create a pandas Dataframe containing the `sourceTable` info for all these `preliminary_visit_image`'s

Now, loop over the `datasetRefs` again, but this time grab the contents of the `sourceTable` table for each `ref` and combine into all into one big pandas DataFrame.  

In [23]:
src_list = []

for i, ref in enumerate(datasetRefs):
    
    # Retrieve sourceTable for this visit & detector...
    dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
    src = butler.get('recalibrated_star_detector', dataId=dataId)
    src_list.append(src.to_pandas())
    print(f"{i} Visit {ref.dataId['visit']}, Detector {ref.dataId['detector']}:  Retrieved catalog of {len(src)} sources.")

src_all = pd.concat(src_list, ignore_index=True)

print("")
print(f"Total combined catalog contains {len(src_all)} sources.")


0 Visit 2024120500122, Detector 5:  Retrieved catalog of 2497 sources.
1 Visit 2024120900310, Detector 5:  Retrieved catalog of 1684 sources.
2 Visit 2024110800288, Detector 4:  Retrieved catalog of 2573 sources.
3 Visit 2024110800251, Detector 5:  Retrieved catalog of 2808 sources.
4 Visit 2024111900094, Detector 3:  Retrieved catalog of 850 sources.
5 Visit 2024111700154, Detector 0:  Retrieved catalog of 1489 sources.
6 Visit 2024120900311, Detector 5:  Retrieved catalog of 1670 sources.
7 Visit 2024120600097, Detector 3:  Retrieved catalog of 2156 sources.
8 Visit 2024112700174, Detector 3:  Retrieved catalog of 2489 sources.
9 Visit 2024112700164, Detector 1:  Retrieved catalog of 1975 sources.
10 Visit 2024120600274, Detector 5:  Retrieved catalog of 2326 sources.
11 Visit 2024120700296, Detector 8:  Retrieved catalog of 2171 sources.
12 Visit 2024120700270, Detector 5:  Retrieved catalog of 1769 sources.
13 Visit 2024121000402, Detector 5:  Retrieved catalog of 696 sources.
14 V

Let's look at the result:

In [24]:
src_all

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,sourceId
0,52.904754,-27.858450,0,2512.106004,23.119362,0.110007,0.117454,52.904754,-27.858450,5.745434e-06,...,False,False,False,False,True,2024120500122,5,r,r_03,600438918259146761
1,52.914526,-27.831140,0,3026.617843,22.112920,0.240403,0.206981,52.914526,-27.831140,1.034801e-05,...,False,False,False,False,True,2024120500122,5,r,r_03,600438918259146762
2,52.915839,-27.828741,0,3074.023404,28.919735,0.100962,0.104703,52.915839,-27.828741,5.135850e-06,...,False,False,False,False,True,2024120500122,5,r,r_03,600438918259146764
3,52.863247,-27.973842,0,336.379056,25.014509,0.715540,0.731315,52.863247,-27.973842,3.586473e-05,...,False,False,False,False,True,2024120500122,5,r,r_03,600438918259146765
4,52.863579,-27.973064,0,351.320595,25.791738,0.513893,0.360174,52.863579,-27.973064,1.852596e-05,...,False,False,False,False,True,2024120500122,5,r,r_03,600438918259146766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307261,53.321904,-27.945975,600452144746202519,244.271520,3938.792488,0.632477,0.639717,53.321904,-27.945975,3.141562e-05,...,False,False,False,False,True,2024120800363,7,z,z_03,600452144746202927
1307262,53.322421,-27.945850,600452144746202519,243.451779,3947.252249,0.766663,0.605894,53.322421,-27.945850,3.092316e-05,...,False,False,False,False,True,2024120800363,7,z,z_03,600452144746202928
1307263,53.298363,-27.890035,600452144746202521,1317.177326,3949.596488,0.010865,0.012085,53.298363,-27.890035,5.872180e-07,...,False,False,False,False,True,2024120800363,7,z,z_03,600452144746202929
1307264,53.299479,-27.889969,600452144746202521,1311.939256,3966.573672,0.401543,0.397382,53.299479,-27.889969,1.957672e-05,...,False,False,False,False,True,2024120800363,7,z,z_03,600452144746202930


#### 3.2.4 Save `src_all` as a CSV file

Let's save `src_all` as a CSV file that we can download and examine with TOPCAT:

***(Rename this file to something else???)***

In [25]:
#src_all.to_csv('LSSTComCam_C26202_fields.csv', index=False)

### 3.3 Extract only those rows containing C26202 from the src_all catalog

In [26]:
# Based on code retrieved from Claude-3.5-Sonnet

# Create a mask to cull sources with "bad" measurements.
mask = (~src_all.pixelFlags_bad) & (~src_all.pixelFlags_saturated) & \
        (~src_all.extendedness_flag) & (src_all.detect_isPrimary)

# Apply mask, keeping only the "good" measurements of `src_all`
src_all_cleaned = src_all[mask]

# Create SkyCoord object for the coordinates of C26202
ref_coord = SkyCoord(ra=raDeg*u.degree, dec=decDeg*u.degree)

# Create SkyCoord object for all points in the dataframe
df_coords = SkyCoord(ra=src_all_cleaned['ra'].values*u.degree, 
                     dec=src_all_cleaned['dec'].values*u.degree)

# Calculate separations
separations = ref_coord.separation(df_coords)

# Create mask for points within 3.0 arcseconds
mask_sep = separations < 3.0*u.arcsec

# Get filtered dataframe
nearby_good_df = src_all_cleaned[mask_sep]

# If you want to include the separations in the result
orig_columns = nearby_good_df.columns
nearby_good_df = src_all_cleaned[mask_sep].copy()
nearby_good_df['separation_c26202'] = separations[mask_sep].arcsec

# Find (and keep) the closet match within the match radius
best_df = nearby_good_df.sort_values('separation_c26202').drop_duplicates(subset=orig_columns, keep='first')


Add magCalib and magCalibErr columns:

In [27]:
# Flux in nano-Janskys to AB magnitudes:
best_df['magCalib'] = -2.5*np.log10(best_df['calibFlux']) + 31.4

# Flux error in nano-Janskys to AB magnitude error:
# Factor of 2.5/math.log(10) is explained here:  https://astronomy.stackexchange.com/questions/38371/how-can-i-calculate-the-uncertainties-in-magnitude-like-the-cds-does
best_df['magCalibErr'] = 2.5/math.log(10)*best_df['calibFluxErr']/best_df['calibFlux']

Display `visit`, `detector`, `band`, `calibFlux`, `calibFluxErr`, `magCalib`, `magCalibErr`, and `separation_c26202` from best_df, sorted by `visit` and `band`:

In [28]:
# Set pandas to show all rows...
pd.set_option("display.max_rows", None)

In [29]:
best_df[['visit', 'detector', 'band', 'calibFlux', 'calibFluxErr', 'magCalib', 'magCalibErr', 'separation_c26202']].sort_values(['visit', 'band'])

,visit,detector,band,calibFlux,calibFluxErr,magCalib,magCalibErr,separation_c26202
1240451,2024110800245,1,i,1.138175e+06,1152.249023,16.259476,0.001099,0.620999
1240452,2024110800245,1,i,5.234673e+03,837.214355,22.102776,0.173649,2.971174
828427,2024110800246,1,r,1.037177e+06,894.923950,16.360367,0.000937,0.618945
744840,2024110800247,5,r,1.033530e+06,947.668457,16.364191,0.000996,0.618540
418956,2024110800248,5,i,1.137089e+06,1207.573730,16.260513,0.001153,0.612946
228168,2024110800249,5,i,1.136692e+06,1169.848877,16.260891,0.001117,0.632081
1116581,2024110800250,5,r,1.036602e+06,922.989136,16.360970,0.000967,0.621688
8564,2024110800251,5,r,1.034454e+06,922.679871,16.363222,0.000968,0.617553
485662,2024110800252,5,i,1.138609e+06,1163.647461,16.259064,0.001110,0.620816
949760,2024110800255,5,r,1.036504e+06,913.186157,16.361073,0.000957,0.614726


In [30]:
print("""Number of rows:  %d""" % (len(best_df['visit'])))

Number of rows:  659


In [31]:
# Reset pandas to its default maximum rows to print to screen
pd.reset_option("display.max_rows")

***Do we need to do any further masking/culling in the above table before proceeding?***

***Save to CSV file???***

***(How to name this file???)***

### 3.4 Match pandas dataframe with observed ComCam magnitudes (best_df) with pandas dataframe with the synthetic magnitudes (df_mags)

In [32]:
# Reset the index to turn the keys into a column
df_mags_reset = df_mags.reset_index()

# Merge the dataframes based on the filter name
combined_df = pd.merge(best_df, df_mags_reset, left_on='band', right_on='index')

combined_df

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,detector,band,physical_filter,sourceId,separation_c26202,magCalib,magCalibErr,index,stiswfcnic_007,mod_008
0,53.137010,-27.863435,0,225.060515,1384.565119,0.015902,0.015486,53.137010,-27.863435,7.619734e-07,...,3,u,u_02,600421334259335417,0.565299,17.623758,0.004159,u,17.572800,17.586964
1,53.137013,-27.863439,0,838.292148,2251.084578,0.004145,0.003785,53.137013,-27.863439,1.927752e-07,...,3,g,g_01,600408133274698727,0.568295,16.702047,0.001081,g,16.691931,16.692687
2,53.137017,-27.863440,0,2627.471672,355.260192,0.019455,0.018823,53.137017,-27.863440,9.279541e-07,...,8,u,u_02,600324579386195996,0.577568,17.614244,0.006147,u,17.572800,17.586964
3,53.137020,-27.863449,0,209.041693,1571.012570,0.013425,0.014802,53.137020,-27.863449,7.282651e-07,...,3,u,u_02,600421330232803631,0.577590,17.629547,0.004220,u,17.572800,17.586964
4,53.137019,-27.863440,0,732.218679,3725.265019,0.004288,0.004606,53.137019,-27.863440,2.256016e-07,...,0,g,g_01,600408131125642749,0.585749,16.701080,0.001116,g,16.691931,16.692687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,53.136434,-27.864230,600452144476193439,3241.705683,1606.638548,0.984166,0.961761,53.136434,-27.864230,4.742024e-05,...,4,z,z_03,600452144476194551,2.955100,21.583492,0.201268,z,16.243679,16.243690
655,53.136432,-27.864232,600355353897992436,1071.845669,748.748902,0.456759,0.465047,53.136432,-27.864232,2.266114e-05,...,3,i,i_06,600355353897993978,2.964292,22.456448,0.374752,i,16.260196,16.259542
656,53.136444,-27.864239,600320187510032299,3992.203634,2723.017264,0.546078,0.346846,53.136444,-27.864239,1.768309e-05,...,1,i,i_06,600320187510033819,2.971174,22.102776,0.173649,i,16.260196,16.259542
657,53.136437,-27.864242,600447734483255776,2069.215691,1232.132208,0.362693,0.457608,53.136437,-27.864242,2.039430e-05,...,2,z,z_03,600447734483256994,2.989717,21.864071,0.284256,z,16.243679,16.243690


In [33]:
print(df_mags)

   stiswfcnic_007    mod_008
u       17.572800  17.586964
g       16.691931  16.692687
r       16.362017  16.361654
i       16.260196  16.259542
z       16.243679  16.243690
y       16.238847  16.238887


In [34]:
# Group by the 'band' column and calculate the median of 'magCalib' for each group
median_values = combined_df.groupby('band')['magCalib'].median().reset_index()
median_values = median_values.rename(columns={'magCalib': 'median_magCalib'})

# Merge the median values back into the combined_df dataframe
combined_df = pd.merge(combined_df, median_values, on='band', how='left')
combined_df

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,band,physical_filter,sourceId,separation_c26202,magCalib,magCalibErr,index,stiswfcnic_007,mod_008,median_magCalib
0,53.137010,-27.863435,0,225.060515,1384.565119,0.015902,0.015486,53.137010,-27.863435,7.619734e-07,...,u,u_02,600421334259335417,0.565299,17.623758,0.004159,u,17.572800,17.586964,17.623758
1,53.137013,-27.863439,0,838.292148,2251.084578,0.004145,0.003785,53.137013,-27.863439,1.927752e-07,...,g,g_01,600408133274698727,0.568295,16.702047,0.001081,g,16.691931,16.692687,16.697416
2,53.137017,-27.863440,0,2627.471672,355.260192,0.019455,0.018823,53.137017,-27.863440,9.279541e-07,...,u,u_02,600324579386195996,0.577568,17.614244,0.006147,u,17.572800,17.586964,17.623758
3,53.137020,-27.863449,0,209.041693,1571.012570,0.013425,0.014802,53.137020,-27.863449,7.282651e-07,...,u,u_02,600421330232803631,0.577590,17.629547,0.004220,u,17.572800,17.586964,17.623758
4,53.137019,-27.863440,0,732.218679,3725.265019,0.004288,0.004606,53.137019,-27.863440,2.256016e-07,...,g,g_01,600408131125642749,0.585749,16.701080,0.001116,g,16.691931,16.692687,16.697416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,53.136434,-27.864230,600452144476193439,3241.705683,1606.638548,0.984166,0.961761,53.136434,-27.864230,4.742024e-05,...,z,z_03,600452144476194551,2.955100,21.583492,0.201268,z,16.243679,16.243690,16.244524
655,53.136432,-27.864232,600355353897992436,1071.845669,748.748902,0.456759,0.465047,53.136432,-27.864232,2.266114e-05,...,i,i_06,600355353897993978,2.964292,22.456448,0.374752,i,16.260196,16.259542,16.260548
656,53.136444,-27.864239,600320187510032299,3992.203634,2723.017264,0.546078,0.346846,53.136444,-27.864239,1.768309e-05,...,i,i_06,600320187510033819,2.971174,22.102776,0.173649,i,16.260196,16.259542,16.260548
657,53.136437,-27.864242,600447734483255776,2069.215691,1232.132208,0.362693,0.457608,53.136437,-27.864242,2.039430e-05,...,z,z_03,600447734483256994,2.989717,21.864071,0.284256,z,16.243679,16.243690,16.244524


In [35]:
# Calculate the number of rows for each filter band
row_counts = combined_df.groupby('band').size().reset_index(name='n_total')

# Merge the row counts back into the combined_df dataframe
combined_df = pd.merge(combined_df, row_counts, on='band', how='left')

combined_df

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,physical_filter,sourceId,separation_c26202,magCalib,magCalibErr,index,stiswfcnic_007,mod_008,median_magCalib,n_total
0,53.137010,-27.863435,0,225.060515,1384.565119,0.015902,0.015486,53.137010,-27.863435,7.619734e-07,...,u_02,600421334259335417,0.565299,17.623758,0.004159,u,17.572800,17.586964,17.623758,31
1,53.137013,-27.863439,0,838.292148,2251.084578,0.004145,0.003785,53.137013,-27.863439,1.927752e-07,...,g_01,600408133274698727,0.568295,16.702047,0.001081,g,16.691931,16.692687,16.697416,165
2,53.137017,-27.863440,0,2627.471672,355.260192,0.019455,0.018823,53.137017,-27.863440,9.279541e-07,...,u_02,600324579386195996,0.577568,17.614244,0.006147,u,17.572800,17.586964,17.623758,31
3,53.137020,-27.863449,0,209.041693,1571.012570,0.013425,0.014802,53.137020,-27.863449,7.282651e-07,...,u_02,600421330232803631,0.577590,17.629547,0.004220,u,17.572800,17.586964,17.623758,31
4,53.137019,-27.863440,0,732.218679,3725.265019,0.004288,0.004606,53.137019,-27.863440,2.256016e-07,...,g_01,600408131125642749,0.585749,16.701080,0.001116,g,16.691931,16.692687,16.697416,165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,53.136434,-27.864230,600452144476193439,3241.705683,1606.638548,0.984166,0.961761,53.136434,-27.864230,4.742024e-05,...,z_03,600452144476194551,2.955100,21.583492,0.201268,z,16.243679,16.243690,16.244524,122
655,53.136432,-27.864232,600355353897992436,1071.845669,748.748902,0.456759,0.465047,53.136432,-27.864232,2.266114e-05,...,i_06,600355353897993978,2.964292,22.456448,0.374752,i,16.260196,16.259542,16.260548,126
656,53.136444,-27.864239,600320187510032299,3992.203634,2723.017264,0.546078,0.346846,53.136444,-27.864239,1.768309e-05,...,i_06,600320187510033819,2.971174,22.102776,0.173649,i,16.260196,16.259542,16.260548,126
657,53.136437,-27.864242,600447734483255776,2069.215691,1232.132208,0.362693,0.457608,53.136437,-27.864242,2.039430e-05,...,z_03,600447734483256994,2.989717,21.864071,0.284256,z,16.243679,16.243690,16.244524,122


In [36]:
# Calculate the differences and add the new columns
#combined_df['offset_stis'] = combined_df['median_magCalib_x'] - combined_df['stiswfcnic_007']
#combined_df['offset_mod'] = combined_df['median_magCalib_x'] - combined_df['mod_008']

#combined_df

**trying again - this time calculate the median then combine the tables for stis and mod**


In [37]:
# Calculate the number of rows for each filter band
row_counts = best_df.groupby('band').size().reset_index(name='n_band')

# Merge the row counts back into the combined_df dataframe
combined_df = pd.merge(best_df, row_counts, on='band', how='left')

combined_df

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,detect_isPrimary,visit,detector,band,physical_filter,sourceId,separation_c26202,magCalib,magCalibErr,n_band
0,53.137010,-27.863435,0,225.060515,1384.565119,0.015902,0.015486,53.137010,-27.863435,7.619734e-07,...,True,2024120100183,3,u,u_02,600421334259335417,0.565299,17.623758,0.004159,31
1,53.137013,-27.863439,0,838.292148,2251.084578,0.004145,0.003785,53.137013,-27.863439,1.927752e-07,...,True,2024112800132,3,g,g_01,600408133274698727,0.568295,16.702047,0.001081,165
2,53.137017,-27.863440,0,2627.471672,355.260192,0.019455,0.018823,53.137017,-27.863440,9.279541e-07,...,True,2024110900199,8,u,u_02,600324579386195996,0.577568,17.614244,0.006147,31
3,53.137020,-27.863449,0,209.041693,1571.012570,0.013425,0.014802,53.137020,-27.863449,7.282651e-07,...,True,2024120100153,3,u,u_02,600421330232803631,0.577590,17.629547,0.004220,31
4,53.137019,-27.863440,0,732.218679,3725.265019,0.004288,0.004606,53.137019,-27.863440,2.256016e-07,...,True,2024112800116,0,g,g_01,600408131125642749,0.585749,16.701080,0.001116,165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,53.136434,-27.864230,600452144476193439,3241.705683,1606.638548,0.984166,0.961761,53.136434,-27.864230,4.742024e-05,...,True,2024120800361,4,z,z_03,600452144476194551,2.955100,21.583492,0.201268,122
655,53.136432,-27.864232,600355353897992436,1071.845669,748.748902,0.456759,0.465047,53.136432,-27.864232,2.266114e-05,...,True,2024111600111,3,i,i_06,600355353897993978,2.964292,22.456448,0.374752,126
656,53.136444,-27.864239,600320187510032299,3992.203634,2723.017264,0.546078,0.346846,53.136444,-27.864239,1.768309e-05,...,True,2024110800245,1,i,i_06,600320187510033819,2.971174,22.102776,0.173649,126
657,53.136437,-27.864242,600447734483255776,2069.215691,1232.132208,0.362693,0.457608,53.136437,-27.864242,2.039430e-05,...,True,2024120700272,2,z,z_03,600447734483256994,2.989717,21.864071,0.284256,122


In [38]:
# Group by the 'n_band' column and calculate the counts of 'band' for each group
count_df = best_df.groupby('band')['magCalib'].count().reset_index()

# Rename the columns for clarity
count_df = count_df.rename(columns={'magCalib': 'n_band'})

count_df

,band,n_band
0,g,165
1,i,126
2,r,188
3,u,31
4,y,26
5,z,122


In [39]:
# Group by the 'band' column and calculate the median of 'magCalib' for each group
median_df = best_df.groupby('band')['magCalib'].median().reset_index()

# Rename the columns for clarity
median_df = median_df.rename(columns={'magCalib': 'median_magCalib'})

median_df

,band,median_magCalib
0,g,16.697416
1,i,16.260548
2,r,16.363602
3,u,17.623758
4,y,16.244087
5,z,16.244524


In [40]:
# Merge the dataframes based on the filter name
combined_df = pd.merge(count_df, median_df, left_on='band', right_on='band')

combined_df

,band,n_band,median_magCalib
0,g,165,16.697416
1,i,126,16.260548
2,r,188,16.363602
3,u,31,17.623758
4,y,26,16.244087
5,z,122,16.244524


In [41]:
# Reset the index to turn the keys into a column
df_mags_reset = df_mags.reset_index()

# Merge the dataframes based on the filter name
combined_df = pd.merge(combined_df, df_mags_reset, left_on='band', right_on='index')

combined_df

,band,n_band,median_magCalib,index,stiswfcnic_007,mod_008
0,g,165,16.697416,g,16.691931,16.692687
1,i,126,16.260548,i,16.260196,16.259542
2,r,188,16.363602,r,16.362017,16.361654
3,u,31,17.623758,u,17.572800,17.586964
4,y,26,16.244087,y,16.238847,16.238887
5,z,122,16.244524,z,16.243679,16.243690


In [42]:
# Calculate the differences and add the new columns
combined_df['offset_stis'] = combined_df['median_magCalib'] - combined_df['stiswfcnic_007']
combined_df['offset_mod'] = combined_df['median_magCalib'] - combined_df['mod_008']

combined_df

,band,n_band,median_magCalib,index,stiswfcnic_007,mod_008,offset_stis,offset_mod
0,g,165,16.697416,g,16.691931,16.692687,0.005485,0.004729
1,i,126,16.260548,i,16.260196,16.259542,0.000352,0.001006
2,r,188,16.363602,r,16.362017,16.361654,0.001584,0.001947
3,u,31,17.623758,u,17.572800,17.586964,0.050958,0.036795
4,y,26,16.244087,y,16.238847,16.238887,0.005240,0.005200
5,z,122,16.244524,z,16.243679,16.243690,0.000845,0.000834


In [43]:
print(combined_df)

  band  n_band  median_magCalib index  stiswfcnic_007    mod_008  offset_stis  \
0    g     165        16.697416     g       16.691931  16.692687     0.005485   
1    i     126        16.260548     i       16.260196  16.259542     0.000352   
2    r     188        16.363602     r       16.362017  16.361654     0.001584   
3    u      31        17.623758     u       17.572800  17.586964     0.050958   
4    y      26        16.244087     y       16.238847  16.238887     0.005240   
5    z     122        16.244524     z       16.243679  16.243690     0.000845   

   offset_mod  
0    0.004729  
1    0.001006  
2    0.001947  
3    0.036795  
4    0.005200  
5    0.000834  


In [44]:
# Define the desired order of 'band'
order = ['u', 'g', 'r', 'i', 'z', 'y']

# Remove the 'index' column
combined_df = combined_df.drop(columns=['index'])

# Reorder the dataframe based on the 'band' column
combined_df['band'] = pd.Categorical(combined_df['band'], categories=order, ordered=True)
combined_df = combined_df.sort_values('band').reset_index(drop=True)

combined_df

,band,n_band,median_magCalib,stiswfcnic_007,mod_008,offset_stis,offset_mod
0,u,31,17.623758,17.572800,17.586964,0.050958,0.036795
1,g,165,16.697416,16.691931,16.692687,0.005485,0.004729
2,r,188,16.363602,16.362017,16.361654,0.001584,0.001947
3,i,126,16.260548,16.260196,16.259542,0.000352,0.001006
4,z,122,16.244524,16.243679,16.243690,0.000845,0.000834
5,y,26,16.244087,16.238847,16.238887,0.005240,0.005200


***Save results to CSV file???***

***(How to name this file???)***

**Let's stop here for now:**

In [45]:
raise StopExecution

## 4. Measure differences between the Observed ComCam and the LSST Synthetic Mags for C26202

***Calculate statistics from the matched dataframe from Section 3 above.***

In [ ]:
# DES DR2 AB offsets based on c26202_stisnic_007.fits from William Wester's DES-doc#15451...

print "AB offsets based on c26202_stisnic_007.fits"
print "==========================================="
print 

aboffset_i = i_wavg - i_ww
aboffset_gr = (g_wavg-r_wavg) - (g_ww-r_ww)
aboffset_ri = (r_wavg-i_wavg) - (r_ww-i_ww)
aboffset_iz = (i_wavg-z_wavg) - (i_ww-z_ww)
aboffset_zY = (z_wavg-Y_wavg) - (z_ww-Y_ww)

print "WAVG offsets"
print "------------"
print """i:    %10.4f""" % (aboffset_i)
print """g-r:  %10.4f""" % (aboffset_gr)
print """r-i:  %10.4f""" % (aboffset_ri)
print """i-z:  %10.4f""" % (aboffset_iz)
print """z-Y:  %10.4f""" % (aboffset_zY)
print 

aboffset_i = i_wavg - i_ww
aboffset_gr = (g_auto-r_auto) - (g_ww-r_ww)
aboffset_ri = (r_auto-i_auto) - (r_ww-i_ww)
aboffset_iz = (i_auto-z_auto) - (i_ww-z_ww)
aboffset_zY = (z_auto-Y_auto) - (z_ww-Y_ww)

print "MAG_AUTO offsets"
print "----------------"
print """i:    %10.4f""" % (aboffset_i)
print """g-r:  %10.4f""" % (aboffset_gr)
print """r-i:  %10.4f""" % (aboffset_ri)
print """i-z:  %10.4f""" % (aboffset_iz)
print """z-Y:  %10.4f""" % (aboffset_zY)
print 


## 5. Sandbox

In [ ]:
datasetRefs = butler.query_datasets("visitSummary", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

#print(datasetRefs)

print(f"\nFound {len(datasetRefs)} preliminary_visit_images")

In [ ]:
# Retrieve sourceTable for this visit & detector...

datasetType = 'sourceTable'
#dataId = {'visit': visit, 'detector': detector}
dataId = ref.dataId['visit']
dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
print(dataId)

src = butler.get(datasetType, dataId=dataId)

print(f"Retrieved catalog of {len(src)} sources.")

In [ ]:
butler.registry.queryDataIds(dimensions=('exposure'))

In [ ]:
icExp = butler.get('icExp', dataId=dataId)

In [ ]:
icExp_info = icExp.getInfo()

In [ ]:
print(icExp_info.getMetadata())

In [ ]:
print(dataId)

In [ ]:
datasetRefs = butler.query_datasets("preliminary_visit_image", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

In [ ]:
for dt in sorted(butler.registry.queryDatasetTypes('*src*')):
    print(dt)

In [ ]:
for dt in sorted(butler.registry.queryDatasetTypes('*icSrc*')):
    print(dt)

In [ ]:
icSrc = butler.get('icSrc', dataId=dataId)

In [ ]:
icSrc.asAstropy()

In [ ]:
src1 = butler.get('src', dataId=dataId)

In [ ]:
src1.asAstropy()